# Develops the Calculation Model for the Online Calculator

In [2]:
import sys
from importlib import reload
import pandas as pd
import numpy as np
# import matplotlib pyplot commands
from matplotlib.pyplot import *
from IPython.display import Image, Markdown
 
# Show Plots in the Notebook
%matplotlib inline
 
# 'style' the plot like fivethirtyeight.com website
style.use('bmh')

In [3]:
rcParams['figure.figsize']= (10, 6)   # set Chart Size
rcParams['font.size'] = 14            # set Font size in Chart

In [4]:
# Access the directory where some utility modules are located in the
# actual heat pump calculator.
sys.path.insert(0, '../../heat-pump-calc/heatpump/')

In [5]:
import library as lib
reload(lib)

<module 'library' from '../../heat-pump-calc/heatpump/library.py'>

In [46]:
# Put the input parameters here for the Calculation Function
city_id = 257
utility_id = 53
hp_model_id = 575

# Existing Heating System Fuel Type
fuel_id = 4



In [47]:
city = lib.city_from_id(city_id)
city

Name                                                             Homer
Latitude                                                       59.6523
Longitude                                                      -151.53
ERHRegionID                                                          2
WAPRegionID                                                          2
FuelRefer                                                        False
FuelCityID                                                         NaN
Oil1Price                                                         2.72
Oil2Price                                                          NaN
PropanePrice                                                       4.1
BirchPrice                                                         NaN
SprucePrice                                                        NaN
CoalPrice                                                          NaN
SteamPrice                                                         NaN
HotWat

In [48]:
hp_model = lib.heat_pump_from_id(hp_model_id)
hp_model

brand                 Fujitsu
ahri_num              8703350
zones                  Single
outdoor_model       AOU12RLS3
indoor_model       ASU12RLS3Y
hspf                       14
in_pwr_5F_max             2.1
capacity_5F_max         16500
in_pwr_47F_min            0.2
Name: 575, dtype: object

In [49]:
utility = lib.util_from_id(utility_id)
utility

ID                                                             53
Name                           Homer Electric Assn  - Residential
Active                                                       True
Type                                                            1
IsCommercial                                                False
ChargesRCC                                                   True
PCE                                                             0
CO2                                                           1.1
CustomerChg                                                    20
DemandCharge                                                  NaN
NameShort                                                  Homer 
Blocks          [(nan, 0.22033000702504113), (nan, nan), (nan,...
Name: 53, dtype: object

In [50]:
fuel = lib.fuel_from_id(fuel_id)
fuel

desc                                                        #1 Oil
unit                                                        gallon
btus                                                        137452
co2                                                          161.3
price_col                                                Oil1Price
effic_choices    [(Standard, 80), (High Efficiency (e.g. Toyost...
Name: 4, dtype: object

In [51]:
# Start a DataFrame to do the hourly analysis.
# First fill it if the outdoor temperature values.
df_tmy = lib.tmy_from_id(city.TMYid)
df_tmy.head()

,db_temp,rh,wind_spd,month
timestamp,,,,
2018-01-01 00:30:00,41.00,67,17.224438,1
2018-01-01 01:30:00,39.92,70,16.105968,1
2018-01-01 02:30:00,42.08,61,17.224438,1
2018-01-01 03:30:00,41.00,67,19.685072,1
2018-01-01 04:30:00,39.92,70,13.869028,1


## Loop through each hour and determine heat load supplied by the Heat Pump

Things that limit heat supplied by the heat pump:
* Heat Distribution issues
* Low temperature cutoff
* Maximum heat output of heat pump

In [60]:
fuel_effic = 0.8

low_temp_cutoff = 5.0    # temperature below which heat pump doesn't run.

bldg_floor_area = 2100.0   # ft2
ua_home = 0.18 * bldg_floor_area     # Btu/hr/deg-F: will be calculated at some point; is not an input
garage_car_count = 1    # 2 means 2-car garage.
ua_garage = 70.0 * garage_car_count     # need better model based on construction of building.

# Percentages of the home floor area (including garage) in different
# zones.
hp_zone_pct = 0.4
hp_adjacent_pct = 0.4
hp_separate_pct = 0.2
doors_open_to_adjacent = False    # Are doors open to zones adjacent to the heat pump
only_hp = False      # maximum input power of the heat pump in kW

df_hourly = df_tmy.reset_index()[['db_temp']]
df_hourly['day_of_year'] = df_tmy.index.dayofyear
df_hourly['month'] =  df_tmy.index.month

# Determine days that the heat pump is running.  Look at the minimum
# temperature for the day, and ensure that it is above the low 
# temperature cutoff.
hp_is_running = lambda x: (x.min() > low_temp_cutoff)
df_hourly['running'] = df_hourly.groupby('day_of_year')['db_temp'].transform(hp_is_running)

# Calculate the COP of the heat pump for every hour of the year.
df_hourly['cop'] =  lib.cop_from_temp_out(df_hourly.db_temp, hp_model.hspf)

max_in_pwr = hp_model.in_pwr_5F_max

# need to better calculate these
balance_point_home = 60.0   
balance_point_garage = 50.0

In [61]:
# BTU loads in the hour for the heat pump and for the secondary system.
hp_load = []
secondary_load = []

for _, h in df_hourly.iterrows():
    # calculate total heat load for the hour.
    # Really need to recognize that delta-T to outdoors is lower in the adjacent and remote spaces
    # if there heat pump is the only source of heat.
    total_load = max(0.0, balance_point_home - h.db_temp) * ua_home + max(0.0, balance_point_garage - h.db_temp) * ua_garage
    if not h.running:
        hp_load.append(0.0)
        secondary_load.append(total_load)
    else:
        max_hp_output = max_in_pwr * h.cop * 3412.
        if only_hp:
            hp_ld = min(total_load, max_hp_output)
            hp_load.append(hp_ld)
        else:
            # Nowhere near correct yet.  Just get a calc framework working.
            hp_ld = min(total_load * hp_zone_pct, max_hp_output)
            sec_ld = total_load - hp_ld
            hp_load.append(hp_ld)
            secondary_load.append(sec_ld)
            
df_hourly['hp_load'] = hp_load
df_hourly['secondary_load'] =  secondary_load

# calculate kWh use by the heat pump and the Btu use of secondary system.
df_hourly['hp_kwh'] = df_hourly.hp_load / df_hourly.cop / 3412.
df_hourly['secondary_fuel'] = df_hourly.secondary_load / fuel_effic
df_hourly[['hp_load', 'secondary_load', 'hp_kwh', 'secondary_fuel']].sum()

hp_load           2.832394e+07
secondary_load    4.989495e+07
hp_kwh            2.877285e+03
secondary_fuel    6.236869e+07
dtype: float64

In [65]:
df_hourly.describe()

,db_temp,day_of_year,month,cop,hp_load,secondary_load,hp_kwh,secondary_fuel
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,38.797836,183.000000,6.526027,3.076907,3233.326389,5695.770625,0.328457,7119.713281
std,14.336010,105.372043,3.448048,0.319682,2344.474987,4735.792181,0.272269,5919.740226
min,-5.080000,1.000000,1.000000,1.864632,0.000000,0.000000,0.000000,0.000000
25%,28.040000,92.000000,4.000000,2.906105,1212.624000,2268.000000,0.105509,2835.000000
50%,41.000000,183.000000,7.000000,3.178947,2931.264000,4687.200000,0.268329,5859.000000
75%,50.000000,274.000000,10.000000,3.368421,4931.136000,8170.848000,0.487146,10213.560000
max,75.020000,365.000000,12.000000,3.368421,9382.464000,28455.840000,1.231312,35569.800000
